### Merging evaluation systems results

In [ ]:
import os
import path
import pandas as pd

In [10]:
statistics_path = os.path.join(os.sep, "home", "results")
ingestion_statistics = pd.DataFrame()
query_statistics = pd.DataFrame()
for evaluated_system in os.listdir(statistics_path):
    if os.path.isdir(os.path.join(statistics_path, evaluated_system)):
        ingestion_file = os.path.join(statistics_path, evaluated_system, "ingestion_time","ingestion_statistics.csv")
        query_file = os.path.join(statistics_path, evaluated_system, "query_evaluation","statistics.csv")

        ingestion_statistics = pd.concat([ingestion_statistics, pd.read_csv(ingestion_file)], ignore_index=True)
        query_statistics = pd.concat([query_statistics, pd.read_csv(query_file)], ignore_index=True)

ingestion_statistics.to_csv(os.path.join(statistics_path, "ingestion_statistics.csv"), index=False )
query_statistics.to_csv(os.path.join(statistics_path, "query_statistics.csv"), index=False )